In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
     sys.path.insert(0, project_root)

import json
from dotenv import load_dotenv
load_dotenv()
from supabase import create_client

from src.llm.tools.ChunkRetriever import RetrievalService

# ========== Constants and Environment Variables =========
YOUR_APP_DOMAIN = "www.stackifier.com"
TEST_EMAIL = os.environ.get("TEST_EMAIL")
TEST_PASSWORD = os.environ.get("TEST_PASSWORD")
if not TEST_EMAIL or not TEST_PASSWORD:
    raise ValueError("TEST_EMAIL and TEST_PASSWORD must be set in your .env file.")

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_ANON_KEY")
if not supabase_url or not supabase_key:
    raise ValueError("SUPABASE_URL and SUPABASE_ANON_KEY must be set in your .env file.")

auth_client_local = create_client(supabase_url, supabase_key)

print(f"Attempting to sign in with email: {TEST_EMAIL}")
auth_response = auth_client_local.auth.sign_in_with_password(
    {"email": TEST_EMAIL, "password": TEST_PASSWORD}
)
if not auth_response or not auth_response.user:
    error_detail = auth_response.error.message if hasattr(auth_response, 'error') and auth_response.error else "Unknown authentication error"
    raise ConnectionError(f"Supabase authentication failed: {error_detail}. Check credentials and Supabase Auth settings.")
authenticated_user_id_str_local = str(auth_response.user.id)
print(f"Authentication successful. User ID: {authenticated_user_id_str_local}")

user_given_id = authenticated_user_id_str_local

Attempting to sign in with email: user@example.com
Authentication successful. User ID: 372ec112-6fd7-46a7-bab4-abf7623fb05b


In [ ]:
from typing import List
from src.llm.GeminiClient import GeminiClient
from src.llm.tools.FunctionCaller import call_function
from src.llm.tools.PythonCalculatorTool import PythonCalculationTool
from google.genai import types

# Define the function declaration for the model
python_calculator_tool_local = PythonCalculationTool()
python_calculator_decleration = python_calculator_tool_local.get_tool_declaration_data()

# Define the function to execute the Python code
execute_python_calculations = python_calculator_tool_local.execute_python_calculations
all_tools_functions = [
    execute_python_calculations,
]

# Define the function declaration for the model
all_tools_decleration = [
    types.Tool(function_declarations=[python_calculator_decleration]),
    types.Tool(function_declarations=[RetrievalService.get_tool_declaration()])
]

config = {
    "tools": all_tools_decleration,
    "automatic_function_calling": {"disable": True},
    "tool_config": {"function_calling_config": {"mode": "auto"}},
    "thinking_config": {"include_thoughts": True}
}

# prompt = "Show me annual report of tesla in 2021?" # "sum first 50 prime numbers, then divide by 0.04215215"
prompt = "Think first and use the retrieval tool. Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? then divide the value by 0.04215215"
CONV_HISTORY = []
CONV_HISTORY.append(types.Content(role="user", parts=[types.Part(text=prompt)]))
print(f"RAW:{CONV_HISTORY}")
for item in CONV_HISTORY:
    print(item.model_dump_json(indent=4))
client = GeminiClient(config=config)#, model="gemini-2.0-flash")

In [ ]:
from src.helper.llm_helper_chat import create_final_answer_instructions


def get_parts(response):
    return response.candidates[0].content.parts # includes thought and answer, parts size 2.
example_supabase_id = "372ec112-6fd7-46a7-bab4-abf7623fb05b"

# turn_count = 0
# max_turns = 3
# while turn_count < max_turns:
#     turn_count += 1
final_result = None

response_no_stream = client.generate_content(
        contents=CONV_HISTORY,
    )
parts = get_parts(response_no_stream)
CONV_HISTORY.append(types.Content(role="model", parts=parts))
function_call = parts[-1].function_call
function_result = None
if function_call:
    function_result = str(call_function(
        function_call, 
        all_tools_functions, 
        user_supabase_id = example_supabase_id 
        )
    )
    function_response_part = types.Part.from_function_response(
        name=function_call.name,
        response={"output": function_result}
    )
    CONV_HISTORY.append(types.Content(role="model", parts=[function_response_part]))
    
# Create final result:
final_result_instructions = create_final_answer_instructions(prompt, function_result, YOUR_APP_DOMAIN)
CONV_HISTORY.append(types.Content(role="user", parts=[types.Part(text=final_result_instructions)]))
response_no_stream = client.generate_content(
        contents=CONV_HISTORY,
    )
parts = get_parts(response_no_stream)
CONV_HISTORY.append(types.Content(role="user", parts=parts))

In [ ]:
for item in CONV_HISTORY:
    for part in parts:
        print("----Part Start\n")
        #print(part.model_dump_json(indent=4))
        print(f"text: {part.text}")
        if part.function_call:            
            print(f"function_call: {part.function_call.model_dump_json(indent=4)}")
        if part.function_response:
            print(f"function_response: {part.function_response.model_dump_json(indent=4)}")
        print("----Part End----------------------------------------------------------")


In [ ]:
print(parts[-1].function_call)

In [ ]:
if parts[-1].function_call:
    result = str(call_function(
        parts[-1].function_call, 
        all_tools_functions, 
        user_supabase_id = example_supabase_id 
        )
    )
    print(result)

In [9]:
from google import genai
from pydantic import BaseModel
from src.prompts.prompt_manager import PromptManager
QUERY_SPLITTER = PromptManager.get_prompt(
        "query_splitter"
    )

class Split(BaseModel):
    reasoning: str
    task: str
load_dotenv()
client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))
prompt = "Think first and use the retrieval tool. Whats the Gross Carrying Amount for Total intangible assets for tesla and apple in 2021? then divide the value by 0.04215215"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config={
        "response_mime_type": "application/json",
        "response_schema": list[Split],
        "system_instruction" : QUERY_SPLITTER,
    }
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Split] = response.parsed

[
  {"reasoning": "Find the Gross Carrying Amount for Total intangible assets for Tesla in 2021.", "task": "Tesla Total intangible assets Gross Carrying Amount 2021"},
  {"reasoning": "Find the Gross Carrying Amount for Total intangible assets for Apple in 2021.", "task": "Apple Total intangible assets Gross Carrying Amount 2021"}
]
